In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

from keras import losses

Using TensorFlow backend.


In [0]:
# define constants and other set up
image_size = (128, 128, 3)

# set image data format manually to avoid potential confusion
K.set_image_data_format('channels_last')

In [0]:
#perceptual loss
from tensorflow.keras.applications.vgg16 import VGG16
tf.config.experimental_run_functions_eagerly(True)
vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_size)
def perceptual_loss(y_true, y_pred):
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))


In [0]:
#combining pixel and perceptual loss
from tensorflow.keras.applications.vgg16 import VGG16
tf.config.experimental_run_functions_eagerly(True)
vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_size)
def pixel_perceptual_loss(y_true, y_pred):
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    perceptual=K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))
    pixel= ((y_true-y_pred)**2)/(128*128*3)
    combined_loss= (0.5*pixel) + perceptual
    return (combined_loss)

Model

In [0]:
input_img = Input(shape=image_size)  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

x = UpSampling2D((2, 2))(encoded)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = Model(input_img, decoded)
#autoencoder.compile(optimizer='adam', loss=pixel_perceptual_loss, metrics = ['accuracy'])
#autoencoder.compile(optimizer='adam', loss=perceptual_loss, metrics = ['accuracy'])
#autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])
autoencoder.summary()

Model: "model_497"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 64, 64, 32)        0 

Load Dataset

In [0]:
#  pickle_in = open('/content/drive/My Drive/Pickle/pixelated.pickle', 'rb') #lowest level of pixelation
# X1 = pickle.load(pickle_in)
# X2 = pickle.load(pickle_in)
# X3 = pickle.load(pickle_in)
# X4 = pickle.load(pickle_in)
# X5 = pickle.load(pickle_in)
# X6 = pickle.load(pickle_in)
# X = np.concatenate((X1,X2, X3, X4, X5, X6))
# del X1, X2, X3, X4, X5, X6

# pickle_in = open('/content/drive/My Drive/Pickle/pixelated.pickle20', 'rb')
# X1 = pickle.load(pickle_in)
# X2 = pickle.load(pickle_in)
# X3 = pickle.load(pickle_in)
# X4 = pickle.load(pickle_in)
# X5 = pickle.load(pickle_in)
# X6 = pickle.load(pickle_in)
# X = np.concatenate((X1,X2, X3, X4, X5, X6))
# del X1, X2, X3, X4, X5, X6

pickle_in = open('/content/drive/My Drive/Pickle/pixelated.pickle', 'rb')#highest level of pixelation
X1 = pickle.load(pickle_in)
# X2 = pickle.load(pickle_in)
# X3 = pickle.load(pickle_in)
# X4 = pickle.load(pickle_in)
# X5 = pickle.load(pickle_in)
# X6 = pickle.load(pickle_in)
# X = np.concatenate((X1,X2, X3))
# del X1, X2, X3




In [0]:
plt.imshow(X1[5])

In [0]:
pickle_in1 = open('/content/drive/My Drive/Pickle/original.pickle', 'rb')
#y1 = pickle.load(pickle_in1)
#y2 = pickle.load(pickle_in1)
#y4 = pickle.load(pickle_in1)
#y5 = pickle.load(pickle_in1)
#y6 = pickle.load(pickle_in1)
# y = np.concatenate((y1,y2,y3))
# del y1,y2,y3


In [0]:
print(len(y))

6616


Training 

In [0]:
from sklearn.model_selection import train_test_split
print(X.shape)
print(y.shape)

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=42)

print(X_train.shape)
print(y_train.shape)

(6616, 128, 128, 3)
(6616, 128, 128, 3)
(5954, 128, 128, 3)
(5954, 128, 128, 3)


In [0]:
history=autoencoder.fit(X, y, epochs=8, batch_size=50, shuffle=True, validation_data=(X_test, y_test),)

In [0]:
autoencoder.save('/content/drive/My Drive/Trained_models/combined30,again.h5')

In [0]:
from matplotlib import pyplot
pyplot.title('Loss / Combined Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Predict

In [0]:
i = 110
predicted = autoencoder.predict(np.array([X_test[i]]))

plt.imshow(X_test[i]) 
plt.show()
plt.imshow(y_test[i]) # original
plt.show()
plt.imshow(predicted[0]) # predicted
plt.show()


SSIM 

In [0]:
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
predicted = autoencoder.predict(np.array([X_test[i]]))
img=y_test[i]
img_x=X_test[i]
img_noise=predicted[0]
ssim_noise = ssim(img, img_noise,
                  data_range=img_noise.max() - img_noise.min(),multichannel=True)
ssim_noise_new = ssim(img, img_x,
                  data_range=img_x.max() - img_x.min(),multichannel=True)
ssim_noise_original = ssim(img, img,
                  data_range=img.max() - img.min(),multichannel=True)

In [0]:
print (ssim_noise)
print (ssim_noise_new)
print (ssim_noise_original)

PSNR

In [0]:
import numpy 
import math
import cv2
original = y_test[i]
predicted = autoencoder.predict(np.array([X_test[i]]))
contrast = predicted[0]
#contrast = y_test[2]
#contrast = X_test[2]
def psnr(img1, img2):
    mse = numpy.mean((img1 - img2) ** 2 )
    if mse == 0:
      return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d=psnr(original,contrast)
print(d)